In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
import io
from PyPDF2 import PdfReader

In [4]:
#pdf2txt
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [ ]:
import tkinter as tk
from tkinter import filedialog

In [ ]:
import json

In [55]:
# Вытаскиваем PathSupplierLetter – путь к письму поставщика на ПК
# folder_path = 'C:\Users\sokolov\Yandex_Disk\MyData\Projects\loading_into_brand_card_iek\exchange\'
folder_path = 'C:/Users/sokolov/Yandex_Disk/MyData/Projects/loading_into_brand_card_iek/exchange/'
for file in os.listdir(folder_path):
    if '.msg' in file:
#     if file.lower().endswith('.msg'):
#         PathSupplierLetter = os.path.join(folder_path, file)
        PathSupplierLetter = folder_path + file
print(PathSupplierLetter)

C:/Users/sokolov/Yandex_Disk/MyData/Projects/loading_into_brand_card_iek/exchange/Условия по тендеру Т250526-18779 согласованы IEKCRM0002072.msg


In [ ]:
pip install extract_msg

import io
import extract_msg

def extract_attachments_to_memory(PathSupplierLetter):
    """Извлекает вложения из .msg файла и сохраняет .xlsx и .pdf в переменные."""
    xlsx_attachments = []  # Список для хранения .xlsx файлов (в байтах)
    pdf_attachments = []   # Список для хранения .pdf файлов (в байтах)

    # Открываем файл .msg
    msg = extract_msg.Message(PathSupplierLetter)
    
    # Перебираем все вложения
    for attachment in msg.attachments:
        filename = attachment.longFilename  # Имя файла с расширением
        
        # Если вложение - Excel (.xlsx)
        if filename.lower().endswith('.xlsx'):
            file_data = io.BytesIO(attachment.data)  # Читаем данные в память
            xlsx_attachments.append((filename, file_data))
        
        # Если вложение - PDF (.pdf)
        elif filename.lower().endswith('.pdf'):
            file_data = io.BytesIO(attachment.data)  # Читаем данные в память
            pdf_attachments.append((filename, file_data))
    
    msg.close()  # Закрываем файл .msg
    
    return xlsx_attachments, pdf_attachments

specification_xlsx, pdf_attachments = extract_attachments_to_memory(PathSupplierLetter)

In [6]:
specification_xlsx = pd.read_excel('exchange\Т250526-18779.xlsx') #загружаем спецификацию

In [57]:
# Вытаскиваем DiscountEKS – наша скидка
discount_from_base = "Устанавливаемая скидка (скидка от базовой), %"
# Поиск индексов (строка, столбец), где встречается текст
result = np.where(specification_xlsx.apply(lambda x: x.astype(str).str.contains(discount_from_base, regex=False)))
row_idx = result[0][0]  # Индекс строки
col_idx = result[1][0]  # Индекс столбца
DiscountEKS = float(specification_xlsx.iat[row_idx+1, col_idx])
print(DiscountEKS)

57.0


In [ ]:
# # Вытаскиваем текст из письма поставщика
# pdf_path = 'exchange\P250526-19292.pdf' #Путь к письму поставщика

# # print(varton_price.head(5))

# #Функция конвертации pdf2txt
# def extract_text_from_pdf(pdf_path):
#     resource_manager = PDFResourceManager()
#     fake_file_handle = io.StringIO()
#     converter = TextConverter(resource_manager, fake_file_handle)
#     page_interpreter = PDFPageInterpreter(resource_manager, converter)

#     with open(pdf_path, 'rb') as fh:
#         for page in PDFPage.get_pages(fh, 
#                                       caching=True,
#                                       check_extractable=True):
#             page_interpreter.process_page(page)
 
#         text = fake_file_handle.getvalue()
#     # close open handles
#     converter.close()
#     fake_file_handle.close()
#     if text:
#         return text

# # Конвернтируем pdf в txt
# text = extract_text_from_pdf(pdf_path)

In [9]:
with open('exchange\P250526-19292.pdf', 'rb') as file:
    pdf_bytes = file.read()

# Создаем PdfReader из бинарных данных
pdf_reader = PdfReader(io.BytesIO(pdf_bytes))

# Извлекаем текст со всех страниц
pdf_text = ""
for page in pdf_reader.pages:
    pdf_text += page.extract_text() + "\n"

text = pdf_text.strip()
text = text.replace('\n', ' ')
text = text.replace('    ', ' ')
text = text.replace('   ', ' ')
text = text.replace('  ', ' ')

text

'Уважаемые Партнеры! В ответ на запрос о предоставлении разовой скидки компании ООО "СТК СТРОЙ-ВЕК" ИНН 7017247658 в сообщаем, что IEK GROUP предоставляет компании ЭКС Новосибирск (филиал, Томск) в рамках реализации проекта Капитальный ремонт здания АБК право реализации продукции Торговой марки IEK по проекту 20241008-00979 Модернизация энергохозяйства Завод Техно, г. Юрга, ул. 1 -я Железнодорожная, 1 , указанной в предоставленной спецификации в адрес ООО "СТК СТРОЙ-ВЕК" на следующих условиях (с предоставлением следующих скидок): Руководитель отдела продаж по СФО Плахотько К.А. Исх № P250526-19292 От 26.05.2025 ЭКС Новосибирск (филиал, Томск) Вниманию: - Исполнитель: Новикова Екатерина Владимировна Тел: 7 (913) 880-72-46 E-mail: novikovaev@tcr.iek.ru Сумма заказа 364 103,70 рублей с учетом предоставленных спец. условий. Перечень и количество продукции указаны в Приложении Также обращаю Ваше внимание на необходимость сразу же после каждой отгрузки предоставлять в адрес IEK GROUP заверен

In [10]:
# Вытаскиваем DiscountBuyer – скидка клиента
text_discount_index = text.index('По продукции IEK - скидка до ')
text_discount = text[text_discount_index+29:text_discount_index+40]
text_discount = text_discount[:text_discount.index('%')]
text_discount = text_discount.replace(',', '.')
DiscountBuyer = float(text_discount)
print(DiscountBuyer)

52.0


In [11]:
# Вытаскиваем VendorCode – код вендора
vendor_code = "Номер проекта в системе IEK"
# Поиск индексов (строка, столбец), где встречается текст
result = np.where(specification_xlsx.apply(lambda x: x.astype(str).str.contains(vendor_code, regex=False)))
row_idx = result[0][0]  # Индекс строки
col_idx = result[1][0]  # Индекс столбца
VendorCode = specification_xlsx.iat[row_idx, col_idx+1]
print(VendorCode)

20241008-00979


In [12]:
# Вытаскиваем ReceivedCondition – сумма полученных условий
text_order_amount_index = text.index('Сумма заказа ')
text_order_amount = text[text_order_amount_index+12:text_order_amount_index+40]
text_order_amount = text_order_amount[:text_order_amount.index(' рублей с учетом')]
text_order_amount = text_order_amount.replace(' ', '')
text_order_amount = text_order_amount.replace(',', '.')
ReceivedCondition = float(text_order_amount)
print(ReceivedCondition)

364103.7


In [25]:
# Вытаскиваем DateCondition – до какой даты действуют
date_condition = "Конец действия цен"
# Поиск индексов (строка, столбец), где встречается текст
result = np.where(specification_xlsx.apply(lambda x: x.astype(str).str.contains(date_condition, regex=False)))
row_idx = result[0][0]  # Индекс строки
col_idx = result[1][0]  # Индекс столбца
DateCondition = specification_xlsx.iat[row_idx, col_idx+1]
DateCondition = str(DateCondition)
print(DateCondition)

2025-07-25 06:18:12


In [58]:
# Создание главного окна
root = tk.Tk()
root.title("Преференции")

label1 = tk.Label(                      # Создание виджета метки
    text="Выберите тип преференции", # Задание отображаемого текста
    fg="#07e374",                       # Установка цвета текста
    bg="#2f457c",                       # Устанавка фона
    width=60,                           # Установка ширина виджета (в текстовых юнитах)
    height=5                           # Установка высоты виджета (в текстовых юнитах) 
)
label1.pack()  

def preference_nom_exclusive():
    global Preference
    Preference = 1
    return Preference

def preference_nom_on_request():
    global Preference
    Preference = 2
    return Preference

# Создание кнопки
button1 = tk.Button(text="Эксклюзив", command=preference_nom_exclusive)
button1.pack(pady=20)

button1 = tk.Button(text="По запросу", command=preference_nom_on_request)
button1.pack(pady=20)

label2 = tk.Label(                      # Создание виджета метки
    text="После выбора закройте это окно", # Задание отображаемого текста
    fg="#07e374",                       # Установка цвета текста
    bg="#2f457c",                       # Устанавка фона
    width=60,                           # Установка ширина виджета (в текстовых юнитах)
    height=5                           # Установка высоты виджета (в текстовых юнитах) 
)
label2.pack()  

root.mainloop()

print(Preference)

1


In [ ]:
# Данные для сохранения
data = {
    "DiscountEKS": DiscountEKS,
    "DiscountBuyer": DiscountBuyer,
    "Preference": Preference,
    "VendorCode": VendorCode,
    "ReceivedCondition": ReceivedCondition,
    "DateCondition": DateCondition,
    "PathSupplierLetter": PathSupplierLetter
}

# Укажите путь, куда сохранить файл
json_file_path = "exchange/data.json"  # Замените на свой путь

# Сохраняем данные в JSON-файл
with open(json_file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)  # indent для красивого форматирования

Файл сохранён: exchange/data.json
